In [1]:
%load_ext autoreload
%autoreload 2
%config InlineBackend.figure_format = 'retina'

In [2]:
import scanpy as sc 
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import matplotlib.patches as mpatches

In [3]:
import sys 
sys.path.append('../../src')

from spaceoracle.visionary import Visionary
from spaceoracle.plotting.cartography import xy_from_adata
from spaceoracle.plotting.cartography import Cartography


2025-05-04 11:38:45.020643: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /ihome/crc/install/gcc-12.2.0/postgresql/15.2/lib:/ihome/crc/install/gcc-12.2.0/imagemagick/6.9.12/lib:/ihome/crc/install/gcc-12.2.0/geos/3.9.4/lib:/ihome/crc/install/gcc-12.2.0/zstd/zstd-1.5.5/lib:/ihome/crc/install/gcc-12.2.0/unixodbc/2.3.11/lib:/ihome/crc/install/sodium/1.0.18/lib:/ihome/crc/install/gcc-12.2.0/r/4.4.0/lib64/R/lib:/ihome/crc/install/gcc-12.2.0/openmpi/4.1.5/lib:/ihome/crc/install/gcc-12.2.0/fftw/3.3.10/lib:/ihome/crc/install/gcc-12.2.0/glpk/5.0/lib:/ihome/crc/install/gcc-12.2.0/proj/4.9.3/lib:/ihome/crc/install/gcc-12.2.0/gdal/2.4.4/lib:/ihome/crc/install/gcc-12.2.0/netcdf/4.8.1/lib:/ihome/crc/install/gcc-12.2.0/hdf5/1.14.0/lib:/ihome/crc/install/gcc-12.2.0/jags/4.3.2/lib:/ihome/crc/install/gcc-12.2.0/gsl/2.7.1/lib:/ihome/c

In [9]:
test_adata = sc.read_h5ad('/ix/djishnu/shared/djishnu_kor11/training_data_2025/human_lymph_cosmx_309.h5ad')
ref_adata = sc.read_h5ad('/ix/djishnu/shared/djishnu_kor11/training_data_2025/snrna_human_tonsil.h5ad')
ref_models_dir = '/ix/djishnu/shared/djishnu_kor11/lasso_runs/human_tonsil'

ref_adata, test_adata

(AnnData object with n_obs × n_vars = 5778 × 3333
     obs: 'cell_type', 'author_cell_type', 'cell_type_int', 'banksy_celltypes'
     uns: 'cell_thresholds', 'cell_type_int_colors', 'received_ligands', 'received_ligands_tfl'
     obsm: 'spatial', 'spatial_unscaled'
     layers: 'imputed_count', 'normalized_count',
 AnnData object with n_obs × n_vars = 4354 × 6175
     obs: 'cell_type', 'cell_type_orig'
     uns: 'cell_type_colors', 'cell_type_orig_colors', 'log1p'
     obsm: 'X_pca', 'X_umap', 'spatial'
     varm: 'PCs'
     layers: 'normalized_count', 'raw_count'
     obsp: 'connectivities', 'distances')

In [10]:
ref_genes = ref_adata.var_names

test_gex = test_adata.to_df(layer='normalized_count')
test_gex = test_gex.reindex(ref_genes, axis=1).fillna(0)

test_adata = sc.AnnData(
    X = test_gex.values,
    var = pd.DataFrame(index=test_gex.columns),
    obs = test_adata.obs,
    obsm = test_adata.obsm
)

test_adata.layers['imputed_count'] = test_adata.X
test_adata.layers['normalized_count'] = test_adata.X
test_adata

AnnData object with n_obs × n_vars = 4354 × 3333
    obs: 'cell_type', 'cell_type_orig'
    obsm: 'X_pca', 'X_umap', 'spatial'
    layers: 'imputed_count', 'normalized_count'

In [11]:
matching = pd.read_csv('mapping_ct.csv', index_col=0).drop(['test_cell_type', 'ref_cell_type'], axis=1)
matching.head(3)

,reference_cell
c_1_309_10,CAAGGGATCTCATGGA-1
c_1_309_100,CAAGGGATCTCATGGA-1
c_1_309_1000,AGCCACGGTTGCAAGG-1


In [12]:
# some cells are unmapped bc they are cell type not in tonsil
matching = matching.dropna(axis=0)
test_adata = test_adata[matching.index, :]
test_adata

View of AnnData object with n_obs × n_vars = 4033 × 3333
    obs: 'cell_type', 'cell_type_orig'
    obsm: 'X_pca', 'X_umap', 'spatial'
    layers: 'imputed_count', 'normalized_count'

In [ ]:
import warnings
warnings.simplefilter("ignore", category=UserWarning)

renata = Visionary(
    ref_adata=ref_adata,
    test_adata=test_adata,
    ref_json_path = f'{ref_models_dir}/run_params.json',
    prematching=matching,
    matching_annot='cell_type'
)

In [ ]:
# target = 'PAX5'
# gex = 0

# renata.perturb(target, n_propagation=4, save_layer=True)
# gex_df = renata.adata.to_df(layer=f'{target}_4n_{gex}x')
# gex_df.to_parquet(f'/ix/djishnu/shared/djishnu_kor11/genome_screens/human_lymph_cosmx_309/{target}_4n_{gex}x.parquet')

In [ ]:
target = 'FOXO1'
gex = 0

renata.perturb(target, n_propagation=4, save_layer=True)
gex_df = renata.adata.to_df(layer=f'{target}_4n_{gex}x')
gex_df.to_parquet(f'/ix/djishnu/shared/djishnu_kor11/genome_screens/human_lymph_cosmx_309/{target}_4n_{gex}x.parquet')

In [ ]:
from spaceoracle.prophets import Prophet
from spaceoracle.plotting.cartography import xy_from_adata
from spaceoracle.plotting.cartography import Cartography


In [ ]:
color_dict = {
    'T_CD4': "#F721B7",
    'B_germinal_center': "#147EE9",
    'plasma': "#61C17C",
    'B_naive': "#308F0E",
    'B_memory': "#BEB637",
    'Naive CD4 T': "#F721B7",
    'Treg': "#ECC91D",
    'T memory': "#49F12F",
    'Th1': "#C43CF2",
    'Th2': "#2C43F1",
    'T_CD8': "#DD3652",
    'T_follicular_helper': "#E28723",
}

chart = Cartography(renata.adata, color_dict)

In [ ]:
kotarget = 'PAX5'
grid, vec = chart.plot_umap_quiver(
    perturb_target=kotarget,
    grid_scale=1,
    vector_scale=0.15,
    n_neighbors=70,
    figsize=(5, 5), 
    hue='cell_type',
    alpha=0.55,
    scatter_size=30,
    remove_null=True,
    betadata_path='/ix/djishnu/shared/djishnu_kor11/genome_screens/human_lymph_cosmx_309/',
    legend_on_loc=True,
)
plt.title(f'{kotarget} KO')
plt.show()